# 目的変数をstateとする分類問題

In [2]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import SGDClassifier

## データの読み込み

In [15]:
df = pd.read_csv("./ks-projects-201801.csv")
df.head()

,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,usd_pledged_real,usd_goal_real
0,1000002330,The Songs of Adelaide & Abullah,Poetry,Publishing,GBP,2015-10-09,1000.0,2015-08-11 12:12:28,0.0,failed,0,GB,0.0,0.0,1533.95
1,1000003930,Greeting From Earth: ZGAC Arts Capsule For ET,Narrative Film,Film & Video,USD,2017-11-01,30000.0,2017-09-02 04:43:57,2421.0,failed,15,US,100.0,2421.0,30000.00
2,1000004038,Where is Hank?,Narrative Film,Film & Video,USD,2013-02-26,45000.0,2013-01-12 00:20:50,220.0,failed,3,US,220.0,220.0,45000.00
3,1000007540,ToshiCapital Rekordz Needs Help to Complete Album,Music,Music,USD,2012-04-16,5000.0,2012-03-17 03:24:11,1.0,failed,1,US,1.0,1.0,5000.00
4,1000011046,Community Film Project: The Art of Neighborhoo...,Film & Video,Film & Video,USD,2015-08-29,19500.0,2015-07-04 08:35:03,1283.0,canceled,14,US,1283.0,1283.0,19500.00


In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 378661 entries, 0 to 378660
Data columns (total 15 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   ID                378661 non-null  int64  
 1   name              378657 non-null  object 
 2   category          378661 non-null  object 
 3   main_category     378661 non-null  object 
 4   currency          378661 non-null  object 
 5   deadline          378661 non-null  object 
 6   goal              378661 non-null  float64
 7   launched          378661 non-null  object 
 8   pledged           378661 non-null  float64
 9   state             378661 non-null  object 
 10  backers           378661 non-null  int64  
 11  country           378661 non-null  object 
 12  usd pledged       374864 non-null  float64
 13  usd_pledged_real  378661 non-null  float64
 14  usd_goal_real     378661 non-null  float64
dtypes: float64(5), int64(2), object(8)
memory usage: 43.3+ MB


In [18]:
df.describe()

,ID,goal,pledged,backers,usd pledged,usd_pledged_real,usd_goal_real
count,3.786610e+05,3.786610e+05,3.786610e+05,378661.000000,3.748640e+05,3.786610e+05,3.786610e+05
mean,1.074731e+09,4.908079e+04,9.682979e+03,105.617476,7.036729e+03,9.058924e+03,4.545440e+04
std,6.190862e+08,1.183391e+06,9.563601e+04,907.185035,7.863975e+04,9.097334e+04,1.152950e+06
min,5.971000e+03,1.000000e-02,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,1.000000e-02
25%,5.382635e+08,2.000000e+03,3.000000e+01,2.000000,1.698000e+01,3.100000e+01,2.000000e+03
50%,1.075276e+09,5.200000e+03,6.200000e+02,12.000000,3.947200e+02,6.243300e+02,5.500000e+03
75%,1.610149e+09,1.600000e+04,4.076000e+03,56.000000,3.034090e+03,4.050000e+03,1.550000e+04
max,2.147476e+09,1.000000e+08,2.033899e+07,219382.000000,2.033899e+07,2.033899e+07,1.663614e+08


In [19]:
df.isnull().sum()

ID                     0
name                   4
category               0
main_category          0
currency               0
deadline               0
goal                   0
launched               0
pledged                0
state                  0
backers                0
country                0
usd pledged         3797
usd_pledged_real       0
usd_goal_real          0
dtype: int64

目的変数に`state`をセット

とりあえずintのみで計算してみる（欠損の多い`usd pledged`は使わない）

In [22]:
y = df["state"].values
X_01 = df[["goal", "pledged", "backers", "usd_pledged_real", "usd_goal_real"]].values
X_01

array([[1.00000e+03, 0.00000e+00, 0.00000e+00, 0.00000e+00, 1.53395e+03],
       [3.00000e+04, 2.42100e+03, 1.50000e+01, 2.42100e+03, 3.00000e+04],
       [4.50000e+04, 2.20000e+02, 3.00000e+00, 2.20000e+02, 4.50000e+04],
       ...,
       [1.50000e+04, 2.00000e+01, 1.00000e+00, 2.00000e+01, 1.50000e+04],
       [1.50000e+04, 2.00000e+02, 6.00000e+00, 2.00000e+02, 1.50000e+04],
       [2.00000e+03, 5.24000e+02, 1.70000e+01, 5.24000e+02, 2.00000e+03]])

## ロジスティック回帰を行う

In [23]:
clf = SGDClassifier(
    loss = "log",
    penalty = "none",
    max_iter = 10000,
    fit_intercept=True,
    random_state=1234,
    tol=1e-3
)

clf.fit(X_01, y)

SGDClassifier(alpha=0.0001, average=False, class_weight=None,
              early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
              l1_ratio=0.15, learning_rate='optimal', loss='log',
              max_iter=10000, n_iter_no_change=5, n_jobs=None, penalty='none',
              power_t=0.5, random_state=1234, shuffle=True, tol=0.001,
              validation_fraction=0.1, verbose=0, warm_start=False)

## 予測精度を得る

混合行列の作成

In [28]:
from  sklearn.metrics import accuracy_score, precision_recall_fscore_support, confusion_matrix

y_pred = clf.predict(X_01)

正答率

In [29]:
accuracy =  accuracy_score(y, y_pred)
print('正答率（Accuracy） = {:.3f}%'.format(100 * accuracy))

正答率（Accuracy） = 62.042%


Precision, Recall, F1-score

In [30]:
precision, recall, f1_score, _ = precision_recall_fscore_support(y, y_pred)

print('適合率（Precision） = {:.3f}%'.format(100 * precision[1]))
print('再現率（Recall） = {:.3f}%'.format(100 * recall[1]))
print('F1値（F1-score） = {:.3f}%'.format(100 * f1_score[1]))

適合率（Precision） = 79.734%
再現率（Recall） = 42.707%
F1値（F1-score） = 55.622%


/Users/kenjoushunsuke/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


混合行列

In [32]:
conf_mat = confusion_matrix(y, y_pred)
conf_mat = pd.DataFrame(conf_mat)
conf_mat

,0,1,2,3,4,5
0,16594,18535,0,3650,0,0
1,96970,84440,0,16309,0,0
2,1102,962,0,735,0,0
3,0,0,61,133895,0,0
4,526,870,0,450,0,0
5,747,1095,0,1720,0,0
